<a href="https://colab.research.google.com/github/R0N3ldrt/Thesis/blob/main/CNN_boot_v11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traininig NN Spectrum

# Loading Necesary Libraries

In [4]:
# Importing necesary libraries
# Libraries for correct code execution 

import os, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle
import random
import csv
import re
import array
import seaborn as sns
from tqdm import tqdm

from functools import reduce
from random import random, gauss
from math import modf, pi, cos, sin, sqrt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from plotly.subplots import make_subplots
from scipy.signal import savgol_filter
from scipy.stats.stats import pearsonr
from scipy.stats import spearmanr

import scipy.stats as st
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping
from sklearn.mixture import GaussianMixture
import os, time, math, csv, joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.mixture import GaussianMixture

import os, time, math, csv, joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.mixture import GaussianMixture

import warnings
warnings.filterwarnings("ignore")

sns.set_theme()

# Load Enviroment

In [5]:
# BLOQUE PARA USAR DESDE COLAB

# Google drive loading as work station for local-usage of the files.
from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)

#-----------------------------------------------------------------------------

Mounted at /content/gdrive


In [6]:
# Lista para cambiar los paths rapido.
workers = ["Ronald", "Local"]

# Change the number to change the paths.
worker = workers[0]

if worker == "Ronald":
  path = "/content/gdrive/MyDrive/Thesis_Workstation/ANN_dataset"
else: path = os.getcwd()

#New Train/Test Split

In [7]:
input_path = path + "/Spectrum/CNN/new_working_df.csv"

working_df = pd.read_csv(input_path)
working_df['Distance_km'] = working_df['Distance_km'].astype(int)

In [8]:
print(working_df.shape)

(2500, 2073)


In [9]:
excluding_model_data_df = working_df.loc[working_df['PBRS_id'] != 1]

## Get Reference data

In [10]:
model_data_df = working_df.loc[working_df['PBRS_id'] == 1]
model_dict = {}
for row in range(model_data_df.shape[0]):
  distance = working_df.iloc[row,3]
  values = np.array(working_df.iloc[row,6:working_df.shape[1]])
  model_dict[distance] = values

In [11]:
def get_reference_and_sample_data(working_df, num_reference, num_sample):
  reference_arr = [x for x in range(1, num_reference+1)]
  reference_data_df = working_df[working_df['PBRS_id'].isin(reference_arr)]

  sample_arr = [x for x in range(num_reference+1, num_sample+1)]
  sample_data_df = working_df[working_df['PBRS_id'].isin(sample_arr)]
  
  return reference_data_df, sample_data_df

In [12]:
reference_data_df, sample_data_df = get_reference_and_sample_data(working_df, num_reference=50, num_sample=100)

## Get test/train split

In [13]:
def get_idx_train_test_split(working_df, trainingProp = 0.8):
  rows_training = np.array([])
  rows_testing = np.array([])
  distances = [x*80 for x in range(1, 26)]
  for d in distances:
    distance_working_df = working_df.loc[working_df['Distance_km'] == d]
    rows_mixed=np.random.permutation(distance_working_df.shape[0])

    training_amt = math.ceil(distance_working_df.shape[0]*trainingProp)
    testing_amt = distance_working_df.shape[0] - training_amt

    rows_training = np.append(rows_training, rows_mixed[:training_amt])
    rows_testing = np.append(rows_testing, rows_mixed[-testing_amt:])
  
  rows_training = rows_training.astype('int').tolist()
  rows_testing = rows_testing.astype('int').tolist()

  return rows_training, rows_testing

In [14]:
rows_training, rows_testing = get_idx_train_test_split(sample_data_df, trainingProp = 0.8)

## Get frequencies filter

In [15]:
def frequencies_filter(df, num_selected_freq=20):
  info_df = df.iloc[:, 0:6]
  data_df = df.iloc[:, 6:df.shape[1]]
  span_val = int(data_df.shape[1]/num_selected_freq)
  for i in range(num_selected_freq):
    df2 = data_df.iloc[:, (i+1)*span_val].to_frame()
    if i+1 == 1:
      new_df = df2
    else:
      new_df = pd.merge(new_df, df2, left_index=True, right_index=True)
  new_df = info_df.join(new_df)
  
  return new_df

In [16]:
reference_freq_data_df = frequencies_filter(reference_data_df, num_selected_freq=20)

sample_freq_data_df = frequencies_filter(sample_data_df, num_selected_freq=20)

# TEST num_selected_freq values

In [17]:
sample_freq_data_df

,row,PBRS_id,Channels,Distance_km,power_dBm,#span,-28.9375,-25.71875,-22.5,-19.28125,...,3.25,6.46875,9.6875,12.90625,16.125,19.34375,22.5625,25.78125,29.0,32.21875
1250,1250,51,1,80,0,1,-11.371108,-9.888275,-15.896047,-10.401251,...,-11.985474,-10.041763,-10.323638,-17.261711,-10.552016,-16.379117,-18.675457,-11.977936,-11.043479,-15.544026
1251,1251,51,1,160,0,2,-10.211595,-10.649810,-9.031208,-16.839636,...,-12.088256,-11.504632,-7.955859,-11.202520,-8.600032,-10.355622,-9.990740,-8.624055,-16.657460,-18.454892
1252,1252,51,1,240,0,3,-15.204040,-19.299046,-9.498365,-4.519786,...,-18.301708,-13.122521,-7.990432,-9.126269,-17.643167,-13.834385,-15.660303,-6.391696,-9.668572,-9.910335
1253,1253,51,1,320,0,4,-13.629725,-20.440475,-11.564857,-11.154106,...,-14.078396,-7.761546,-11.642466,-15.153048,-17.844676,-10.922279,-9.524572,-12.243487,-12.415649,-30.553415
1254,1254,51,1,400,0,5,-10.817269,-15.656722,-13.972180,-20.629751,...,-18.218828,-15.606157,-8.490232,-17.095917,-7.016890,-12.053797,-11.479498,-6.427090,-17.091879,-19.309437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,2495,100,1,1680,0,21,-10.384048,-15.111328,-10.947651,-15.621388,...,-8.297015,-10.186144,-7.996039,-7.510316,-12.258500,-17.332990,-12.781404,-11.458765,-9.875344,-22.044932
2496,2496,100,1,1760,0,22,-25.989634,-7.263092,-13.035062,-10.594661,...,-11.335702,-11.592846,-9.906562,-7.139437,-11.066623,-11.801799,-12.502789,-7.710519,-16.097928,-19.157402
2497,2497,100,1,1840,0,23,-6.889217,-17.560904,-12.411949,-14.249330,...,-11.040326,-13.328168,-8.388157,-10.133369,-19.487009,-11.853957,-14.058775,-6.946959,-14.632147,-21.732752
2498,2498,100,1,1920,0,24,-17.875031,-8.925251,-11.985465,-14.717908,...,-13.172462,-10.790016,-11.697045,-11.293293,-14.034699,-16.059040,-12.485453,-12.692961,-17.194686,-22.963780


## Get features

In [50]:
#sample_freq_data_df
#distance_reference_df
#selected_rows = rows_training
def get_features(sample_freq_data_df, reference_freq_data_df, selected_rows):
  data_distance = []
  data_mean = []
  data_std = []
  data_pearson_min = []
  data_pearson_mean = []
  data_pearson_max = []
  data_CI_lower = []
  data_CI_upper = []

  for row_idx in tqdm(selected_rows):
    data_Y = sample_freq_data_df.iloc[row_idx,3] # distance of row selected
    data_X = np.array(sample_freq_data_df.iloc[row_idx,6:sample_freq_data_df.shape[1]]) #array of data in row selected
    data_distance.append(data_Y)
    data_std.append(np.std(data_X))
    data_mean.append(np.mean(data_X))

    data_ci = st.t.interval(alpha=0.90, df=len(data_X)-1, loc=np.mean(data_X), scale=st.sem(data_X))
    data_confidence_int_lower = data_ci[0]
    data_confidence_int_upper = data_ci[1]

    # Get reference data
    reference_freq_data_df
    distance_reference_df = reference_freq_data_df.loc[reference_freq_data_df['Distance_km'] == data_Y]

    pearson_vals = []
    confidence_vals_upper = []
    confidence_vals_lower = []
    for reference_row in range(0, distance_reference_df.shape[0]):
      reference_X = np.array(distance_reference_df.iloc[reference_row,6:distance_reference_df.shape[1]])
      pearson_vals.append(round(pearsonr(data_X, reference_X)[0], 5))

      reference_ci = st.t.interval(alpha=0.90, df=len(reference_X)-1, loc=np.mean(reference_X), scale=st.sem(reference_X))
      reference_confidence_int_lower = reference_ci[0]
      confidence_vals_lower.append(reference_confidence_int_lower)
      reference_confidence_int_upper = reference_ci[1]
      confidence_vals_upper.append(reference_confidence_int_upper)

    data_pearson_min.append(np.min(pearson_vals))
    data_pearson_mean.append(np.mean(pearson_vals))    
    data_pearson_max.append(np.max(pearson_vals))

    data_CI_lower.append(abs(np.min(confidence_vals_lower)-data_confidence_int_lower))
    data_CI_upper.append(abs(np.max(confidence_vals_upper)-data_confidence_int_upper))
  data = {'distance':data_distance,
                  'mean':data_mean,
                  'std':data_std,
                  'pearson_min':data_pearson_min,
                  'pearson_mean':data_pearson_mean,
                  'pearson_max':data_pearson_max,
                  'delta_CI_min':data_CI_lower,
                  'delta_CI_max':data_CI_upper}

  data_df = pd.DataFrame(data)

  return data_df

In [51]:
training_df = get_features(sample_freq_data_df, reference_freq_data_df, rows_training)

testing_df = get_features(sample_freq_data_df, reference_freq_data_df, rows_testing)

100%|██████████| 250/250 [00:13<00:00, 18.02it/s]


In [33]:
'''
def get_features(excluding_model_data_df, selected_rows):

  data_distance = []
  data_mean = []
  data_std = []
  data_pearson_min = []
  data_pearson_mean = []
  data_pearson_max = []

  for row_idx in tqdm(selected_rows):
    data_Y = excluding_model_data_df.iloc[row_idx,3]
    data_X = np.array(excluding_model_data_df.iloc[row_idx,6:excluding_model_data_df.shape[1]])
    data_distance.append(data_Y)
    data_std.append(np.std(data_X))
    data_mean.append(np.mean(data_X))

    pearson_vals = []
    for k, v in model_dict.items():  
      pearson_vals.append(round(pearsonr(v, data_X)[0], 5))
      print(v)
      print(data_X)
      break
    data_pearson_min.append(np.min(pearson_vals))
    data_pearson_mean.append(np.mean(pearson_vals))    
    data_pearson_max.append(np.max(pearson_vals))
    break
  data = {'distance':data_distance,
                  'mean':data_mean,
                  'std':data_std,
                  'pearson_min':data_pearson_min,
                  'pearson_mean':data_pearson_mean,
                  'pearson_max':data_pearson_max}

  data_df = pd.DataFrame(data)

  return data_df
training_df = get_features(excluding_model_data_df, rows_training)

testing_df = get_features(excluding_model_data_df, rows_testing)
'''

# Bootstrapping

In [ ]:
input_path = path + "/Spectrum/CNN/working_df.csv"

working_df = pd.read_csv(input_path)
working_df['Distance_km'] = working_df['Distance_km'].astype(int)

In [ ]:
distances = [x*80 for x in range(1, 26)]

## Get max and min values of each column

In [ ]:
def get_max_min_val_col(working_df):
  data_working_df = working_df.iloc[:, 6:working_df.shape[1]]
  h_list = list(data_working_df.columns.values)
  max_min_col_dict = {header:[] for header in h_list}

  for d in tqdm(distances):
    boot_dist_df = working_df.loc[working_df['Distance_km'] == d]
    for idx in range(6, boot_dist_df.shape[1]):
      max_val_col = float(boot_dist_df.iloc[:, [idx]].max()) # max val in col
      min_val_col = float(boot_dist_df.iloc[:, [idx]].min()) # min val in col
      h = boot_dist_df.iloc[:, [idx]].columns # header of col
      h = h[0]
      max_min_col_dict[h].append((d, min_val_col, max_val_col))
  return max_min_col_dict

In [ ]:
max_min_col_dict = get_max_min_val_col(working_df)

In [ ]:
print(max_min_col_dict)

## Create new samples using bootstrap technique

In [ ]:
def create_bootstrap_samples(working_df, max_min_col_dict, num_of_new_samples, previos_amt_of_samples=13):
  import random

  row_data = []
  PBRS_id_data = []
  Distance_km_data = []
  power_dBm_data = []
  span_data = []

  h_list = list(working_df.columns.values)
  new_data = {header:[] for header in h_list}


  last_row_in_old_df_val = working_df.shape[0]
  for new_sample in tqdm(range(num_of_new_samples)): # generating (num_of_new_samples) new samples for en data
    for d in distances:
      new_data['row'].append(last_row_in_old_df_val)
      last_row_in_old_df_val += 1
      new_data['PBRS_id'].append(new_sample+(previos_amt_of_samples+1))
      new_data['Channels'].append(1)

      new_data['Distance_km'].append(d)
      new_data['power_dBm'].append(0)
      new_data['#span'].append(int(d/80))

      for k, v in max_min_col_dict.items(): # header:(d, min_val_col, max_val_col)
        for val in v:
          if val[0] == d:
            bootstrap_val = round(random.uniform(val[1], val[2]), 14)
            new_data[str(k)].append(bootstrap_val)
            break
  new_data_df = pd.DataFrame.from_dict(new_data)

  return new_data_df

In [ ]:
new_data_df = create_bootstrap_samples(working_df, max_min_col_dict, num_of_new_samples=87, previos_amt_of_samples=13)

## Combine previos df with bootstraped df

In [ ]:
new_working_df = working_df.append(new_data_df)

In [ ]:
new_working_df.to_csv(path+'/Spectrum/CNN/new_working_df.csv', index=False)

# Neural Network

## Pre-processing

In [47]:
#training_df=pd.read_csv(path+'/Spectrum/CNN/training_data_DNN.csv')

Y = training_df.iloc[:, 0].to_numpy().reshape(-1,1)
X = training_df.iloc[:, 1:training_df.shape[1]]

sc_input = MinMaxScaler()
sc_output = MinMaxScaler()
Y_train = sc_output.fit_transform(Y) # convert distances to values from 0 to 1
X_train = sc_input.fit_transform(X) # convert features to values from 0 to 1

In [48]:
#testing_df=pd.read_csv(path+'/Spectrum/CNN/testing_data_DNN.csv')

Y = testing_df.iloc[:, 0].to_numpy().reshape(-1,1)
X = testing_df.iloc[:, 1:testing_df.shape[1]]


sc_input = MinMaxScaler()
sc_output = MinMaxScaler()
Y_test = sc_output.fit_transform(Y) # convert distances to values from 0 to 1
X_test = sc_input.fit_transform(X) # convert features to values from 0 to 1

## Model

In [49]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping


def ann():
    model = Sequential()
    model.add(Dense(76, input_dim = 7, activation = 'tanh'))
    model.add(Dense(38,activation='tanh'))
    model.add(Dense(19,activation='tanh'))
    model.add(Dense(1))
    model.compile(loss ='mean_squared_error',optimizer = 'RMSprop')
    return model


model_ann = KerasRegressor(build_fn=ann,epochs=5000,batch_size=32)

start_time = time.time()
callback = EarlyStopping(monitor='loss', patience=500)
model_ann.fit(X_train, Y_train, callbacks=[callback])
time_train_ann = time.time() - start_time

Epoch 1/5000
32/32 [==============================] - 1s 2ms/step - loss: 0.1169
Epoch 2/5000
32/32 [==============================] - 0s 2ms/step - loss: 0.0970
Epoch 3/5000
32/32 [==============================] - 0s 2ms/step - loss: 0.0920
Epoch 4/5000
32/32 [==============================] - 0s 2ms/step - loss: 0.0917
Epoch 5/5000
32/32 [==============================] - 0s 2ms/step - loss: 0.0874
Epoch 6/5000
32/32 [==============================] - 0s 2ms/step - loss: 0.0861
Epoch 7/5000
32/32 [==============================] - 0s 2ms/step - loss: 0.0836
Epoch 8/5000
32/32 [==============================] - 0s 2ms/step - loss: 0.0791
Epoch 9/5000
32/32 [==============================] - 0s 2ms/step - loss: 0.0752
Epoch 10/5000
32/32 [==============================] - 0s 2ms/step - loss: 0.0704
Epoch 11/5000
32/32 [==============================] - 0s 2ms/step - loss: 0.0683
Epoch 12/5000
32/32 [==============================] - 0s 2ms/step - loss: 0.0649
Epoch 13/5000
32/32 [====

KeyboardInterrupt: ignored

## Test Model

In [ ]:
start_time = time.time()
Y_test_pred=model_ann.predict(X_test)
pred=list(Y_test_pred)

time_eval_ann=time.time()-start_time

dist_min=sc_output.data_min_[0] # 80
dist_max=sc_output.data_max_[0] # 2000

real=list(list(zip(*Y_test))[0])
real_abs=np.add(dist_min,np.multiply((dist_max-dist_min),real)) # convert back to distances values from scalar
real_abs=[int(np.round(i)) for i in real_abs]
pred_abs=np.add(dist_min,np.multiply((dist_max-dist_min),pred)) # convert back to distances values from scalar

error=np.divide(np.abs(np.subtract(np.array(real_abs),np.array(pred_abs))),np.array(real_abs))
error_v2 = np.abs(np.subtract(np.array(real_abs),np.array(pred_abs)))
res=pd.DataFrame({"dist":real_abs,"pred":pred_abs,"error":error})
display(res)
res_v2=pd.DataFrame({"dist":real_abs,"pred":pred_abs,"error":error_v2})
display(res_v2)
#res.to_csv(path+"/Spectrum/CNN/results_pme_supervisedFeatures.csv", header=True, index=False)
plt.plot(real_abs,error,'bo')
plt.show()
plt.plot(real_abs,error_v2,'bo')
plt.show()

# Testing bunch of neural networks

## Test 1 fail

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping


def ann():
    model = Sequential()
    model.add(Dense(100, input_dim = 7, activation = 'tanh'))
    model.add(Dense(50,activation='tanh'))
    model.add(Dense(25,activation='tanh'))
    model.add(Dense(1))
    model.compile(loss ='mean_squared_error',optimizer = 'RMSprop')
    return model


model_ann = KerasRegressor(build_fn=ann,epochs=5000,batch_size=32, verbose=0)

start_time = time.time()
callback = EarlyStopping(monitor='loss', patience=500)
model_ann.fit(X_train, Y_train, callbacks=[callback])
time_train_ann = time.time() - start_time

start_time = time.time()
Y_test_pred=model_ann.predict(X_test)
pred=list(Y_test_pred)

time_eval_ann=time.time()-start_time

dist_min=sc_output.data_min_[0] # 80
dist_max=sc_output.data_max_[0] # 2000

real=list(list(zip(*Y_test))[0])
real_abs=np.add(dist_min,np.multiply((dist_max-dist_min),real)) # convert back to distances values from scalar
real_abs=[int(np.round(i)) for i in real_abs]
pred_abs=np.add(dist_min,np.multiply((dist_max-dist_min),pred)) # convert back to distances values from scalar

error=np.divide(np.abs(np.subtract(np.array(real_abs),np.array(pred_abs))),np.array(real_abs))
error_v2 = np.abs(np.subtract(np.array(real_abs),np.array(pred_abs)))
res=pd.DataFrame({"dist":real_abs,"pred":pred_abs,"error":error})
display(res)
res_v2=pd.DataFrame({"dist":real_abs,"pred":pred_abs,"error":error_v2})
display(res_v2)
#res.to_csv(path+"/Spectrum/CNN/results_pme_supervisedFeatures.csv", header=True, index=False)
plt.plot(real_abs,error,'bo')
plt.show()
plt.plot(real_abs,error_v2,'bo')
plt.show()

## Test 2 fail

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping


def ann():
    model = Sequential()
    model.add(Dense(76, input_dim = 7, activation = 'tanh'))
    model.add(Dense(38,activation='tanh'))
    model.add(Dense(19,activation='tanh'))
    model.add(Dense(1))
    model.compile(loss ='mean_squared_error',optimizer = 'RMSprop')
    return model


model_ann = KerasRegressor(build_fn=ann,epochs=5000,batch_size=32, verbose=0)

start_time = time.time()
callback = EarlyStopping(monitor='loss', patience=500)
model_ann.fit(X_train, Y_train, callbacks=[callback])
time_train_ann = time.time() - start_time

start_time = time.time()
Y_test_pred=model_ann.predict(X_test)
pred=list(Y_test_pred)

time_eval_ann=time.time()-start_time

dist_min=sc_output.data_min_[0] # 80
dist_max=sc_output.data_max_[0] # 2000

real=list(list(zip(*Y_test))[0])
real_abs=np.add(dist_min,np.multiply((dist_max-dist_min),real)) # convert back to distances values from scalar
real_abs=[int(np.round(i)) for i in real_abs]
pred_abs=np.add(dist_min,np.multiply((dist_max-dist_min),pred)) # convert back to distances values from scalar

error=np.divide(np.abs(np.subtract(np.array(real_abs),np.array(pred_abs))),np.array(real_abs))
error_v2 = np.abs(np.subtract(np.array(real_abs),np.array(pred_abs)))
res=pd.DataFrame({"dist":real_abs,"pred":pred_abs,"error":error})
display(res)
res_v2=pd.DataFrame({"dist":real_abs,"pred":pred_abs,"error":error_v2})
display(res_v2)
#res.to_csv(path+"/Spectrum/CNN/results_pme_supervisedFeatures.csv", header=True, index=False)
plt.plot(real_abs,error,'bo')
plt.show()
plt.plot(real_abs,error_v2,'bo')
plt.show()

## Test 3 better

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping


def ann():
    model = Sequential()
    model.add(Dense(256, input_dim = 7, activation = 'tanh'))
    model.add(Dense(128,activation='tanh'))
    model.add(Dense(64,activation='tanh'))
    model.add(Dense(32,activation='tanh'))
    model.add(Dense(1))
    model.compile(loss ='mean_squared_error',optimizer = 'RMSprop')
    return model


model_ann = KerasRegressor(build_fn=ann,epochs=5000,batch_size=32, verbose=0)

start_time = time.time()
callback = EarlyStopping(monitor='loss', patience=500)
model_ann.fit(X_train, Y_train, callbacks=[callback])
time_train_ann = time.time() - start_time

start_time = time.time()
Y_test_pred=model_ann.predict(X_test)
pred=list(Y_test_pred)

time_eval_ann=time.time()-start_time

dist_min=sc_output.data_min_[0] # 80
dist_max=sc_output.data_max_[0] # 2000

real=list(list(zip(*Y_test))[0])
real_abs=np.add(dist_min,np.multiply((dist_max-dist_min),real)) # convert back to distances values from scalar
real_abs=[int(np.round(i)) for i in real_abs]
pred_abs=np.add(dist_min,np.multiply((dist_max-dist_min),pred)) # convert back to distances values from scalar

error=np.divide(np.abs(np.subtract(np.array(real_abs),np.array(pred_abs))),np.array(real_abs))
error_v2 = np.abs(np.subtract(np.array(real_abs),np.array(pred_abs)))
res=pd.DataFrame({"dist":real_abs,"pred":pred_abs,"error":error})
display(res)
res_v2=pd.DataFrame({"dist":real_abs,"pred":pred_abs,"error":error_v2})
display(res_v2)
#res.to_csv(path+"/Spectrum/CNN/results_pme_supervisedFeatures.csv", header=True, index=False)
plt.plot(real_abs,error,'bo')
plt.show()
plt.plot(real_abs,error_v2,'bo')
plt.show()

## Test 4

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping


def ann():
    model = Sequential()
    model.add(Dense(256, input_dim = 7, activation = 'tanh'))
    model.add(Dense(256,activation='tanh'))
    model.add(Dense(128,activation='tanh'))
    model.add(Dense(128,activation='tanh'))
    model.add(Dense(64,activation='tanh'))
    model.add(Dense(64,activation='tanh'))
    model.add(Dense(32,activation='tanh'))
    model.add(Dense(1))
    model.compile(loss ='mean_squared_error',optimizer = 'RMSprop')
    return model


model_ann = KerasRegressor(build_fn=ann,epochs=5000,batch_size=32, verbose=0)

start_time = time.time()
callback = EarlyStopping(monitor='loss', patience=500)
model_ann.fit(X_train, Y_train, callbacks=[callback])
time_train_ann = time.time() - start_time

start_time = time.time()
Y_test_pred=model_ann.predict(X_test)
pred=list(Y_test_pred)

time_eval_ann=time.time()-start_time

dist_min=sc_output.data_min_[0] # 80
dist_max=sc_output.data_max_[0] # 2000

real=list(list(zip(*Y_test))[0])
real_abs=np.add(dist_min,np.multiply((dist_max-dist_min),real)) # convert back to distances values from scalar
real_abs=[int(np.round(i)) for i in real_abs]
pred_abs=np.add(dist_min,np.multiply((dist_max-dist_min),pred)) # convert back to distances values from scalar

error=np.divide(np.abs(np.subtract(np.array(real_abs),np.array(pred_abs))),np.array(real_abs))
error_v2 = np.abs(np.subtract(np.array(real_abs),np.array(pred_abs)))
res=pd.DataFrame({"dist":real_abs,"pred":pred_abs,"error":error})
display(res)
res_v2=pd.DataFrame({"dist":real_abs,"pred":pred_abs,"error":error_v2})
display(res_v2)
#res.to_csv(path+"/Spectrum/CNN/results_pme_supervisedFeatures.csv", header=True, index=False)
plt.plot(real_abs,error,'bo')
plt.show()
plt.plot(real_abs,error_v2,'bo')
plt.show()

## Test 5

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping


def ann():
    model = Sequential()
    model.add(Dense(256, input_dim = 7, activation = 'tanh'))
    model.add(Dense(256,activation='tanh'))
    model.add(Dense(128,activation='tanh'))
    model.add(Dense(128,activation='tanh'))
    model.add(Dense(64,activation='tanh'))
    model.add(Dense(64,activation='tanh'))
    model.add(Dense(32,activation='tanh'))
    model.add(Dense(1))
    model.compile(loss ='mean_squared_error',optimizer = 'RMSprop')
    return model


model_ann = KerasRegressor(build_fn=ann,epochs=5000,batch_size=32, verbose=0)

start_time = time.time()
callback = EarlyStopping(monitor='loss', patience=500)
model_ann.fit(X_train, Y_train, callbacks=[callback])
time_train_ann = time.time() - start_time

start_time = time.time()
Y_test_pred=model_ann.predict(X_test)
pred=list(Y_test_pred)

time_eval_ann=time.time()-start_time

dist_min=sc_output.data_min_[0] # 80
dist_max=sc_output.data_max_[0] # 2000

real=list(list(zip(*Y_test))[0])
real_abs=np.add(dist_min,np.multiply((dist_max-dist_min),real)) # convert back to distances values from scalar
real_abs=[int(np.round(i)) for i in real_abs]
pred_abs=np.add(dist_min,np.multiply((dist_max-dist_min),pred)) # convert back to distances values from scalar

error=np.divide(np.abs(np.subtract(np.array(real_abs),np.array(pred_abs))),np.array(real_abs))
error_v2 = np.abs(np.subtract(np.array(real_abs),np.array(pred_abs)))
res=pd.DataFrame({"dist":real_abs,"pred":pred_abs,"error":error})
display(res)
res_v2=pd.DataFrame({"dist":real_abs,"pred":pred_abs,"error":error_v2})
display(res_v2)
#res.to_csv(path+"/Spectrum/CNN/results_pme_supervisedFeatures.csv", header=True, index=False)
plt.plot(real_abs,error,'bo')
plt.show()
plt.plot(real_abs,error_v2,'bo')
plt.show()

# Test 6 batch size 64

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping


def ann():
    model = Sequential()
    model.add(Dense(256, input_dim = 7, activation = 'tanh'))
    model.add(Dense(128,activation='tanh'))
    model.add(Dense(64,activation='tanh'))
    model.add(Dense(32,activation='tanh'))
    model.add(Dense(1))
    model.compile(loss ='mean_squared_error',optimizer = 'RMSprop')
    return model


model_ann = KerasRegressor(build_fn=ann,epochs=5000,batch_size=64, verbose=0)

start_time = time.time()
callback = EarlyStopping(monitor='loss', patience=500)
model_ann.fit(X_train, Y_train, callbacks=[callback])
time_train_ann = time.time() - start_time

start_time = time.time()
Y_test_pred=model_ann.predict(X_test)
pred=list(Y_test_pred)

time_eval_ann=time.time()-start_time

dist_min=sc_output.data_min_[0] # 80
dist_max=sc_output.data_max_[0] # 2000

real=list(list(zip(*Y_test))[0])
real_abs=np.add(dist_min,np.multiply((dist_max-dist_min),real)) # convert back to distances values from scalar
real_abs=[int(np.round(i)) for i in real_abs]
pred_abs=np.add(dist_min,np.multiply((dist_max-dist_min),pred)) # convert back to distances values from scalar

error=np.divide(np.abs(np.subtract(np.array(real_abs),np.array(pred_abs))),np.array(real_abs))
error_v2 = np.abs(np.subtract(np.array(real_abs),np.array(pred_abs)))
res=pd.DataFrame({"dist":real_abs,"pred":pred_abs,"error":error})
display(res)
res_v2=pd.DataFrame({"dist":real_abs,"pred":pred_abs,"error":error_v2})
display(res_v2)
#res.to_csv(path+"/Spectrum/CNN/results_pme_supervisedFeatures.csv", header=True, index=False)
plt.plot(real_abs,error,'bo')
plt.show()
plt.plot(real_abs,error_v2,'bo')
plt.show()

# Test 7 batch size 128

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping


def ann():
    model = Sequential()
    model.add(Dense(256, input_dim = 7, activation = 'tanh'))
    model.add(Dense(128,activation='tanh'))
    model.add(Dense(64,activation='tanh'))
    model.add(Dense(32,activation='tanh'))
    model.add(Dense(1))
    model.compile(loss ='mean_squared_error',optimizer = 'RMSprop')
    return model


model_ann = KerasRegressor(build_fn=ann,epochs=5000,batch_size=128, verbose=0)

start_time = time.time()
callback = EarlyStopping(monitor='loss', patience=500)
model_ann.fit(X_train, Y_train, callbacks=[callback])
time_train_ann = time.time() - start_time

start_time = time.time()
Y_test_pred=model_ann.predict(X_test)
pred=list(Y_test_pred)

time_eval_ann=time.time()-start_time

dist_min=sc_output.data_min_[0] # 80
dist_max=sc_output.data_max_[0] # 2000

real=list(list(zip(*Y_test))[0])
real_abs=np.add(dist_min,np.multiply((dist_max-dist_min),real)) # convert back to distances values from scalar
real_abs=[int(np.round(i)) for i in real_abs]
pred_abs=np.add(dist_min,np.multiply((dist_max-dist_min),pred)) # convert back to distances values from scalar

error=np.divide(np.abs(np.subtract(np.array(real_abs),np.array(pred_abs))),np.array(real_abs))
error_v2 = np.abs(np.subtract(np.array(real_abs),np.array(pred_abs)))
res=pd.DataFrame({"dist":real_abs,"pred":pred_abs,"error":error})
display(res)
res_v2=pd.DataFrame({"dist":real_abs,"pred":pred_abs,"error":error_v2})
display(res_v2)
#res.to_csv(path+"/Spectrum/CNN/results_pme_supervisedFeatures.csv", header=True, index=False)
plt.plot(real_abs,error,'bo')
plt.show()
plt.plot(real_abs,error_v2,'bo')
plt.show()

# Test 8

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow.keras import regularizers

def ann():
    model = Sequential()
    model.add(Dense(80, input_dim=7, activation='tanh', kernel_regularizer=regularizers.l2(0.001), kernel_initializer='he_uniform'))
    model.add(Dense(80, activation='tanh', kernel_regularizer=regularizers.l2(0.001)))
    model.add(Dense(40, activation='tanh', kernel_regularizer=regularizers.l2(0.001)))
    model.add(Dense(40, activation='tanh', kernel_regularizer=regularizers.l2(0.001)))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss ='mean_squared_error',optimizer = 'RMSprop')
    return model


model_ann = KerasRegressor(build_fn=ann,epochs=5000,batch_size=32, verbose=0)

start_time = time.time()
callback = EarlyStopping(monitor='loss', patience=500)
model_ann.fit(X_train, Y_train, callbacks=[callback])
time_train_ann = time.time() - start_time

start_time = time.time()
Y_test_pred=model_ann.predict(X_test)
pred=list(Y_test_pred)

time_eval_ann=time.time()-start_time

dist_min=sc_output.data_min_[0] # 80
dist_max=sc_output.data_max_[0] # 2000

real=list(list(zip(*Y_test))[0])
real_abs=np.add(dist_min,np.multiply((dist_max-dist_min),real)) # convert back to distances values from scalar
real_abs=[int(np.round(i)) for i in real_abs]
pred_abs=np.add(dist_min,np.multiply((dist_max-dist_min),pred)) # convert back to distances values from scalar

error=np.divide(np.abs(np.subtract(np.array(real_abs),np.array(pred_abs))),np.array(real_abs))
error_v2 = np.abs(np.subtract(np.array(real_abs),np.array(pred_abs)))
res=pd.DataFrame({"dist":real_abs,"pred":pred_abs,"error":error})
display(res)
res_v2=pd.DataFrame({"dist":real_abs,"pred":pred_abs,"error":error_v2})
display(res_v2)
#res.to_csv(path+"/Spectrum/CNN/results_pme_supervisedFeatures.csv", header=True, index=False)
plt.plot(real_abs,error,'bo')
plt.show()
plt.plot(real_abs,error_v2,'bo')
plt.show()

# Test 9

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping


def ann():
    model = Sequential()
    model.add(Dense(256, input_dim = 7, activation = 'tanh'))
    model.add(Dense(256,activation='tanh'))
    model.add(Dense(128,activation='tanh'))
    model.add(Dense(128,activation='tanh'))
    model.add(Dense(64,activation='tanh'))
    model.add(Dense(64,activation='tanh'))
    model.add(Dense(32,activation='tanh'))
    model.add(Dense(1))
    model.compile(loss ='mean_squared_error',optimizer = 'RMSprop')
    return model


model_ann = KerasRegressor(build_fn=ann,epochs=5000,batch_size=128, verbose=0)

start_time = time.time()
callback = EarlyStopping(monitor='loss', patience=500)
model_ann.fit(X_train, Y_train, callbacks=[callback])
time_train_ann = time.time() - start_time

start_time = time.time()
Y_test_pred=model_ann.predict(X_test)
pred=list(Y_test_pred)

time_eval_ann=time.time()-start_time

dist_min=sc_output.data_min_[0] # 80
dist_max=sc_output.data_max_[0] # 2000

real=list(list(zip(*Y_test))[0])
real_abs=np.add(dist_min,np.multiply((dist_max-dist_min),real)) # convert back to distances values from scalar
real_abs=[int(np.round(i)) for i in real_abs]
pred_abs=np.add(dist_min,np.multiply((dist_max-dist_min),pred)) # convert back to distances values from scalar

error=np.divide(np.abs(np.subtract(np.array(real_abs),np.array(pred_abs))),np.array(real_abs))
error_v2 = np.abs(np.subtract(np.array(real_abs),np.array(pred_abs)))
res=pd.DataFrame({"dist":real_abs,"pred":pred_abs,"error":error})
display(res)
res_v2=pd.DataFrame({"dist":real_abs,"pred":pred_abs,"error":error_v2})
display(res_v2)
#res.to_csv(path+"/Spectrum/CNN/results_pme_supervisedFeatures.csv", header=True, index=False)
plt.plot(real_abs,error,'bo')
plt.show()
plt.plot(real_abs,error_v2,'bo')
plt.show()